NOTE: ARCHITECTURES ARE PRELIMINARY AND HAVEN'T BEEN DEBUGGED FOR LAYER INPUT/OUTPUT INCONSISTENCIES

IN OTHER WORDS: **THIS SPECIFIC NOTEBOOK IS NOT COMPLETE. DO NOT REFER TO IT.**

In [ ]:
import numpy as np                  # just in case
import pandas as pd                 # for extracting info

import torch                        
import torch.nn as nn               
import torch.nn.functional as F
from torchsummary import summary        

In [ ]:
# global variables and parameters 
img_size = 256              # assuming square images -> subject to change
img_channels = 3           # assuming RGB images 
num_outputs = 1             # we want a single number signifying the rating

# Data Imports

The Yelp Database (uploaded to Drive) is extracted to a pandas dataframe for input into the proposed neural networks shown below.

# A Simple CNN

## Architecture

In [ ]:
conv_kernel_size = 3        
pool_kernel_size = 4

In [ ]:
# as a preliminary, we start with a super-basic CNN with pooling 
class SimpleCNN(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(img_channels, 8, conv_kernel_size)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(8, 16, conv_kernel_size)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.fc1 = nn.Linear(16*(256/pool_kernel_size**2)**2, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))       # each conv layer consists of convolution -> activation func -> pooling
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)                     # to feed into linear layers
        x = self.fc1(x)
        x = self.fc2(x)             

        return x        # no non-linearity applied since we want lin. regression

## Training

# CNN + Dropout

## Architecture

In [ ]:
dropout_rate_1 = 0.2
dropout_rate_2 = 0.4
pool_kernel_size = 2

In [ ]:
# borrowing from a pytorch tutorial + modifying to get a single linear output
class DropoutCNN(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(img_channels, 8, conv_kernel_size)
        self.conv2 = nn.Conv2d(8, 16, conv_kernel_size)
        self.dropout1 = nn.Dropout2d(dropout_rate_1)
        self.dropout2 = nn.Dropout2d(dropout_rate_2)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.fc1 = nn.Linear(16*(256/pool_kernel_size)**2, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)

        x = self.conv2(x)
        x = F.relu(x)

        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        return x

# Pretrained Model Performance

## Class Setup

## Training

# Custom Embedding CNN

This notebook defines the components of a custom CNN architecture inspired by the semantic embedding to review prediction pipeline detailed by Tang et. al. (https://www.ijcai.org/Proceedings/15/Papers/193.pdf)